In [1]:
import pymysql
import pandas as pd
from tqdm import tqdm
from openpyxl import load_workbook


## Function to connect to MySQL server

In [2]:
# Set up the database connection parameters
db_params = {
    'host': '127.0.0.1',
    'port': 3308,
    'user': 'ksingh',
    'password': 'c-8t!2H9StisTE0ethlF',
    'database': 'ethermail_bi'
}

# Establish the database connection
conn = pymysql.connect(**db_params)

# Create a cursor object
cursor = conn.cursor()

In [3]:
def connect_mysql(sql_query):
    cursor.execute(sql_query)
    df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
    return df


In [4]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

def save_dataframe_to_gsheet(df, sheet_name):
    json_credentials_path = '/Users/Singh/Ethermail/Analysis/credentials.json'
    # Set up credentials
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
            "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

    creds = ServiceAccountCredentials.from_json_keyfile_name(json_credentials_path, scope)
    client = gspread.authorize(creds)

    # Open the spreadsheet
    spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/15l06Hkj5INfjUym0QU7FAzoJoS4XwUVpg7g65fnsGBM/edit?usp=sharing")

    # Check if the sheet exists
    try:
        worksheet = spreadsheet.worksheet(sheet_name)
        # Clear the existing values
        worksheet.clear()
        DEFAULT_ROWS = 1000
        DEFAULT_COLS = 26
        worksheet.resize(rows=DEFAULT_ROWS, cols=DEFAULT_COLS)
    except gspread.exceptions.WorksheetNotFound:
        # Create a new sheet if it doesn't exist
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows="1000", cols="50")


    # Replace non-standard floating-point values with "NaN" string (or None or any placeholder you prefer)
    df = df.replace([float('inf'), float('-inf')], 'NaN')
    df.fillna('NaN', inplace=True)

    # Convert the entire DataFrame to a list of lists, including headers
    all_values = [df.columns.tolist()] + df.values.tolist()
    
    # Resize the worksheet and update the entire worksheet in one go
    worksheet.update('A1', all_values)

In [5]:
aff_list = pd.read_csv('/Users/Singh/Ethermail/Analysis/Affliate/Affiliate Payout v2.csv')

In [6]:
aff_list

,affiliate email/twitter,affiliate ID,with tx,no tx,comment
0,silvanconcept@gmail.com,637353091b036693c524d7e5,NaN,NaN,NaN
1,learnfromchris@gmail.com,642aea08dd6bd18b4098e641,19.0,37.0,NaN
2,karasb7778@gmail.com,6457a7047b34021b83303915,NaN,NaN,NaN
3,0xc9eb02dc80eb24e988f1481ab0093c199a545b6d@eth...,63f591fe64f69a1bda048343,NaN,NaN,NaN
4,everyours550@gmail.com,644d15ae639cb1f4d233267e,NaN,NaN,NaN
5,cryptosnipermoonshot@gmail.com,641dbb6021c689ab3cf27911,NaN,NaN,NaN
6,bibinprasannanmpt@gmail.com,6421455be135da64b23a0388,NaN,NaN,already paid something
7,caoquyenlinh95@gmail.com,64243c19104e0f075512c3be,NaN,NaN,NaN
8,mayowa.owo@gmail.com,64242fd7d7963271db19f013,5.0,11.0,NaN
9,contact@coineasy.xyz,640c8263939b0891f089dc34,NaN,NaN,already paid something


## Query for affliates users without transaction + flagged accounts removed 
### Users without bots and no transactions
	SELECT 
		DATE_FORMAT(uq.created_at, '%Y-%m') AS time_month,
		COUNT(DISTINCT(uq.user_id)) AS count_users_trans
	FROM 
		ethermail_bi.user_query uq
	LEFT JOIN 
		ethermail_bi.flagged_accounts fa 
	ON 
		fa.user_id = uq.user_id 
	WHERE 
		uq.channel_user_id = '6457a7047b34021b83303915'
	AND 
		uq.onboarding_complete = 1
	AND 
		(fa.user_id IS NULL
		OR
		uq.has_transactions = 1)
	GROUP BY 
		DATE_FORMAT(uq.created_at, '%Y-%m');

In [28]:
#df_transaction = connect_mysql(f"SELECT DATE_FORMAT(uq.created_at, '%Y-%m') AS time_month, COUNT(DISTINCT(uq.user_id)) AS count_users_trans FROM ethermail_bi.user_query uq LEFT JOIN ethermail_bi.flagged_accounts fa ON fa.user_id = uq.user_id WHERE uq.channel_user_id = '{affliate_name}' AND uq.onboarding_complete = 1 AND (fa.user_id IS NULL OR uq.has_transactions = 1) GROUP BY DATE_FORMAT(uq.created_at, '%Y-%m');")

## Query for affliates users with transaction  + flagged accounts removed 
### Users without bots and has transactions
	SELECT 
		DATE_FORMAT(uq.created_at, '%Y-%m') AS time_month, -- Time in Months
		COUNT(DISTINCT(uq.user_id)) AS count_users_trans -- User count 
	FROM 
		ethermail_bi.user_query uq
	LEFT JOIN 
		ethermail_bi.flagged_accounts fa 
	ON 
		fa.user_id = uq.user_id 
	WHERE 
		uq.channel_user_id = '6305e8e6c72123a1d2ee7e9d' -- Sample ID
	AND 
		uq.onboarding_complete = 1 -- Onboarded users only
	AND 
		(fa.user_id IS NULL -- Either bots
		OR
		uq.has_transactions = 1) -- Or have completed transaction
	GROUP BY 
		DATE_FORMAT(uq.created_at, '%Y-%m'); -- Grouping for months

In [29]:
#df_no_transaction = connect_mysql(f"SELECT DATE_FORMAT(uq.created_at, '%Y-%m') AS time_month, COUNT(DISTINCT(uq.user_id)) AS count_users_no_trans FROM ethermail_bi.user_query uq LEFT JOIN ethermail_bi.flagged_accounts fa ON fa.user_id = uq.user_id WHERE uq.channel_user_id = '{affliate_name}' AND uq.onboarding_complete = 1 AND fa.user_id IS NULL AND uq.has_transactions = 0 GROUP BY DATE_FORMAT(uq.created_at, '%Y-%m');")

In [7]:
# Remove all spaces from the 'affiliate ID' column
aff_list['affiliate ID'] = aff_list['affiliate ID'].str.replace(' ', '', regex=False)
aff_list = aff_list[['affiliate email/twitter','affiliate ID']]

In [8]:
aff_list

,affiliate email/twitter,affiliate ID
0,silvanconcept@gmail.com,637353091b036693c524d7e5
1,learnfromchris@gmail.com,642aea08dd6bd18b4098e641
2,karasb7778@gmail.com,6457a7047b34021b83303915
3,0xc9eb02dc80eb24e988f1481ab0093c199a545b6d@eth...,63f591fe64f69a1bda048343
4,everyours550@gmail.com,644d15ae639cb1f4d233267e
5,cryptosnipermoonshot@gmail.com,641dbb6021c689ab3cf27911
6,bibinprasannanmpt@gmail.com,6421455be135da64b23a0388
7,caoquyenlinh95@gmail.com,64243c19104e0f075512c3be
8,mayowa.owo@gmail.com,64242fd7d7963271db19f013
9,contact@coineasy.xyz,640c8263939b0891f089dc34


In [9]:
#Create the first sheet for values
aff_list.to_excel("/Users/Singh/Ethermail/Analysis/Affliate/my_excel_file.xlsx", sheet_name = "Sheet0" ,index=False)

save_dataframe_to_gsheet(aff_list, "Sheet1")

# Specify a writer
with pd.ExcelWriter('my_excel_file.xlsx', engine='openpyxl', mode='a') as writer:
    for index, row in tqdm(aff_list.iterrows(), total=len(aff_list), desc="Processing rows"):
        affiliate_name = row['affiliate ID']

        # Dataframe for each affiliate with transaction and no bots
        df_transaction = connect_mysql(f"SELECT DATE_FORMAT(uq.created_at, '%Y-%m') AS time_month, COUNT(DISTINCT(uq.user_id)) AS count_users_trans FROM ethermail_bi.user_query uq WHERE uq.channel_user_id = '{affiliate_name}' AND uq.onboarding_complete = 1 AND uq.invite_level = 1 AND uq.has_transactions = 1 GROUP BY DATE_FORMAT(uq.created_at, '%Y-%m');")

        # Dataframe for each affiliate with no transaction and no bots
        df_no_transaction = connect_mysql(f"SELECT DATE_FORMAT(uq.created_at, '%Y-%m') AS time_month, COUNT(DISTINCT(uq.user_id)) AS count_users_no_trans FROM ethermail_bi.user_query uq LEFT JOIN ethermail_bi.flagged_accounts fa ON fa.user_id = uq.user_id WHERE uq.channel_user_id = '{affiliate_name}' AND uq.onboarding_complete = 1 AND uq.invite_level = 1 AND fa.user_id IS NULL AND uq.has_transactions = 0 GROUP BY DATE_FORMAT(uq.created_at, '%Y-%m');")

        # Merge the two tables together
        df_merge = pd.merge(df_transaction, df_no_transaction, on='time_month', how='left')

        # Create a sheet name, ensuring it is valid and unique
        sheet_name = str(row["affiliate email/twitter"])[:31]  # Max length = 31 chars
        sheet_name = ''.join(x if x.isalnum() else "_" for x in sheet_name)  # Keep alnum chars only


        # Check and adjust if sheet name already exists
        try:
            writer.book[sheet_name]
            suffix = 1
            new_sheet_name = f"{sheet_name[:28]}_{suffix}"
            while new_sheet_name in writer.book.sheetnames:
                suffix += 1
                new_sheet_name = f"{sheet_name[:28]}_{suffix}"
            sheet_name = new_sheet_name
        except KeyError:
            pass  # Sheet name is available, proceed with writing

        # Write DataFrame to a specific sheet
        df_merge.to_excel(writer, sheet_name=sheet_name, index=False)

        #save_dataframe_to_gsheet(df_merge, sheet_name)


/Users/Singh/Ethermail/Analysis/.venv/lib/python3.10/site-packages/gspread/worksheet.py:1046: UserWarning: [Deprecated][in version 6.0.0]: method signature will change to: 'Worksheet.update(value = [[]], range_name=)' arguments 'range_name' and 'values' will swap, values will be mandatory of type: 'list(list(...))'
  warnings.warn(
Processing rows: 100%|██████████| 28/28 [19:11<00:00, 41.13s/it]


## Test data for certain ID for validation

In [33]:
affliate_name_1 = "62fb694c7e150cafb0d033d7"

# Datframe for each affliate with transaction and no bots
df_transaction = connect_mysql(f"SELECT DATE_FORMAT(uq.created_at, '%Y-%m') AS time_month, COUNT(DISTINCT(uq.user_id)) AS count_users_trans FROM ethermail_bi.user_query uq WHERE uq.channel_user_id = '{affliate_name_1}' AND uq.onboarding_complete = 1 AND uq.has_transactions = 1 GROUP BY DATE_FORMAT(uq.created_at, '%Y-%m');")

# Datframe for each affliate with no transaction and no bots
df_no_transaction = connect_mysql(f"SELECT DATE_FORMAT(uq.created_at, '%Y-%m') AS time_month, COUNT(DISTINCT(uq.user_id)) AS count_users_no_trans FROM ethermail_bi.user_query uq LEFT JOIN ethermail_bi.flagged_accounts fa ON fa.user_id = uq.user_id WHERE uq.invited_by = '{affliate_name_1}' AND uq.onboarding_complete = 1 AND fa.user_id IS NULL AND uq.has_transactions = 0 GROUP BY DATE_FORMAT(uq.created_at, '%Y-%m');")

# Merge the two tables together
df_merge = pd.merge(df_transaction, df_no_transaction, on='time_month', how='left')
df_merge


,time_month,count_users_trans,count_users_no_trans
0,2022-09,164,4.0
1,2022-10,509,2.0
2,2022-11,1844,3.0
3,2022-12,92,NaN
4,2023-01,209,4.0
5,2023-02,101,NaN
6,2023-03,1607,131.0
7,2023-04,280,9.0
8,2023-05,40,2.0
9,2023-06,21,1.0


### Check if the afid exists in the channels table
    SELECT 
        c.user_id,
        c.name,
        c.channel_type,
        CASE 
            WHEN c.user_id IS NOT NULL THEN 1
            ELSE 0
        END AS value_exists
    FROM ethermail_bi.channels c
    WHERE c.user_id = '6457a7047b34021b83303915';

In [34]:
df_exists_1 = connect_mysql('''SELECT 
    c.user_id,
    c.name,
    c.channel_type,
    CASE 
        WHEN c.user_id IS NOT NULL THEN 1
        ELSE 0
    END AS value_exists
FROM ethermail_bi.channels c
WHERE c.user_id = '6457a7047b34021b83303915';''')
df_exists_1

,user_id,name,channel_type,value_exists
0,6457a7047b34021b83303915,influencer_karasb,MARKETING,1


In [35]:
# Initialize an empty dataframe to store all results
final_df = pd.DataFrame(columns= ['user_id', 'name', 'channel_type', 'value_exists'])

for index, row in tqdm(aff_list.iterrows(), total=len(aff_list), desc="Processing rows"):
    affiliate_name = row['affiliate ID']

   # Using a parameterized query to avoid SQL injection
    df_exists = connect_mysql(f"SELECT c.user_id, c.name, c.channel_type, CASE WHEN c.user_id IS NOT NULL THEN 1 ELSE 0 END AS value_exists FROM ethermail_bi.channels c WHERE c.user_id = '{affiliate_name}';")
    
    # Append df_exists to the final dataframe
    final_df = pd.concat([final_df, df_exists], ignore_index=True)



Processing rows: 100%|██████████| 28/28 [00:01<00:00, 18.45it/s]


In [36]:
final_df

,user_id,name,channel_type,value_exists
0,637353091b036693c524d7e5,influencer_silvanconcept,MARKETING,1
1,642aea08dd6bd18b4098e641,influencer_learnfromchris,MARKETING,1
2,6457a7047b34021b83303915,influencer_karasb,MARKETING,1
3,63f591fe64f69a1bda048343,influencer_0xc9eb02dc80eb24e988f1481ab0093c199...,MARKETING,1
4,644d15ae639cb1f4d233267e,influencer_everyours550@gmail.com,MARKETING,1
5,641dbb6021c689ab3cf27911,influencer_cryptosnipermoonshot,MARKETING,1
6,6421455be135da64b23a0388,influencer_bibinprasannanmpt,MARKETING,1
7,64243c19104e0f075512c3be,influencer_caoquyenlinh95,MARKETING,1
8,64242fd7d7963271db19f013,influencer_mayowa.owo,MARKETING,1
9,640c8263939b0891f089dc34,influencer_coineasy,MARKETING,1


In [37]:
final_df.merge(aff_list, how='outer', left_on='user_id', right_on='affiliate ID')

,user_id,name,channel_type,value_exists,affiliate email/twitter,affiliate ID
0,637353091b036693c524d7e5,influencer_silvanconcept,MARKETING,1,silvanconcept@gmail.com,637353091b036693c524d7e5
1,642aea08dd6bd18b4098e641,influencer_learnfromchris,MARKETING,1,learnfromchris@gmail.com,642aea08dd6bd18b4098e641
2,6457a7047b34021b83303915,influencer_karasb,MARKETING,1,karasb7778@gmail.com,6457a7047b34021b83303915
3,63f591fe64f69a1bda048343,influencer_0xc9eb02dc80eb24e988f1481ab0093c199...,MARKETING,1,0xc9eb02dc80eb24e988f1481ab0093c199a545b6d@eth...,63f591fe64f69a1bda048343
4,644d15ae639cb1f4d233267e,influencer_everyours550@gmail.com,MARKETING,1,everyours550@gmail.com,644d15ae639cb1f4d233267e
5,641dbb6021c689ab3cf27911,influencer_cryptosnipermoonshot,MARKETING,1,cryptosnipermoonshot@gmail.com,641dbb6021c689ab3cf27911
6,6421455be135da64b23a0388,influencer_bibinprasannanmpt,MARKETING,1,bibinprasannanmpt@gmail.com,6421455be135da64b23a0388
7,64243c19104e0f075512c3be,influencer_caoquyenlinh95,MARKETING,1,caoquyenlinh95@gmail.com,64243c19104e0f075512c3be
8,64242fd7d7963271db19f013,influencer_mayowa.owo,MARKETING,1,mayowa.owo@gmail.com,64242fd7d7963271db19f013
9,640c8263939b0891f089dc34,influencer_coineasy,MARKETING,1,contact@coineasy.xyz,640c8263939b0891f089dc34


In [38]:
final_df.to_excel('/Users/Singh/Ethermail/Analysis/Affliate/affiliate_check.xlsx', index=False)